In [40]:
import numpy as np
from collections import Counter

In [2]:
np.random.choice(5,5, replace=False)

array([1, 2, 0, 4, 3])

In [3]:
np.random.choice([1,2,3,4,5,6,7], 5, replace=False)

array([4, 5, 1, 6, 2])

In [7]:
X = np.linspace(0,10,11)
X

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [8]:
np.argwhere(X >= 5)

array([[ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10]])

In [9]:
np.argwhere(X >=5).flatten()

array([ 5,  6,  7,  8,  9, 10])

In [4]:
def entropy(y):
    hist = np.bincount(y)
    Ps = hist / len(y)
    return -np.sum([p*np.log2(p) for p in Ps if p>0]) 

In [5]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [41]:
class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_features=None):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _grow_tree(self, X, y, depth=0):
        N, n_features = X.shape
        n_labels = len(np.unique(y))

        # checking the stopping criteria:
        if(depth >= self.max_depth
          or N < self.min_sample_split
          or n_labels == 1):
            leaf_value = self._most_common(y)
            return Node(value = leaf_value)

        feature_idxs = np.random.choice(n_features, self.n_features, replace=False)
        
        best_feature, best_threshold = self._best_criteria(X, y, feature_idxs)
        
        left_idxs, right_idxs = self._split(X[:, best_feature], y, best_threshold)
        
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(best_feature, best_threshold, left, right)

    def _best_criteria(self, X, y, feature_idxs):
        split_feature, split_threshold = None, None
        best_gain = -1
        for idx in feature_idxs:
            X_column = X[:, idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_feature = idx
                    split_threshold = threshold
        return split_feature, split_threshold

    def _information_gain(self, y, X_column, threshold):
        entropy_parent = entropy(y)

        left_idxs, right_idxs = self._split(X_column, y, threshold)

        entropy_child = (len(left_idxs) / len(y))*(entropy(y[left_idxs])) + (len(right_idxs) / len(y))*(entropy(y[right_idxs]))

        return entropy_parent - entropy_child

    def _split(self, X_column, y, threshold):
        left = np.argwhere(X_column <= threshold).flatten()
        right = np.argwhere(X_column > threshold).flatten()
        return left, right
                    
    def _most_common(self, y):
        most_common = Counter(y).most_common(1)
        return most_common[0][0]

In [42]:
from sklearn import datasets

In [43]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [50]:
X.shape

(569, 30)

In [51]:
y.shape

(569,)

In [52]:
type(X)

numpy.ndarray

In [53]:
type(y)

numpy.ndarray

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [46]:
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

In [47]:
y_pred = clf.predict(X_test)

In [48]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [49]:
acc = accuracy(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9298245614035088
